Circos plot visualization to show active ligand-target links between 
interacting cells 
================ 
Robin Browaeys 
3-7-2019 
 
<!-- github markdown built using  
rmarkdown::render("vignettes/circos.Rmd", output_format = "github_document") 
--> 
 
This vignette shows how NicheNet can be used to predict active 
ligand-target links between multiple interacting cells and how you can 
make a circos plot to summarize the top-predicted links (via the 
circlize package). This vignette starts in the same way as the main, 
basis, NicheNet vignette [NicheNet’s ligand activity analysis on a gene 
set of interest: predict active ligands and their target 
genes](ligand_activity_geneset.md):`vignette("ligand_activity_geneset", 
package="nichenetr")`. Make sure you understand the different steps 
described in that vignette before proceeding with this vignette. In 
contrast to the basic vignette, we will look communication between 
multiple cell types. More specifically, we will predict which ligands 
expressed by both CAFs and endothelial cells can induce the p-EMT 
program in neighboring malignant cells (See Puram et al. 2017). 
 
### Load packages required for this vignette 
 


In [ ]:
library(nichenetr) 
library(tidyverse) 
library(circlize) 


 
### Read in expression data of interacting cells 
 
First, we will read in the publicly available single-cell data from 
CAFs, endothelial cells and malignant cells from HNSCC 
tumors. 
 


In [ ]:
hnscc_expression = readRDS(url("https://zenodo.org/record/3260758/files/hnscc_expression.rds")) 
expression = hnscc_expression$expression 
sample_info = hnscc_expression$sample_info # contains meta-information about the cells 


 
Secondly, we will determine which genes are expressed in CAFs, 
endothelial and malignant cells from high quality primary tumors. 
Therefore, we wil not consider cells from tumor samples of less quality 
or from lymph node metastases. To determine expressed genes, we use the 
definition used by of Puram et 
al. 
 


In [ ]:
tumors_remove = c("HN10","HN","HN12", "HN13", "HN24", "HN7", "HN8","HN23") 
 
CAF_ids = sample_info %>% filter(`Lymph node` == 0 & !(tumor %in% tumors_remove) & `non-cancer cell type` == "CAF") %>% pull(cell) 
endothelial_ids = sample_info %>% filter(`Lymph node` == 0 & !(tumor %in% tumors_remove) & `non-cancer cell type` == "Endothelial") %>% pull(cell) 
malignant_ids = sample_info %>% filter(`Lymph node` == 0 & !(tumor %in% tumors_remove) & `classified  as cancer cell` == 1) %>% pull(cell) 
 
expressed_genes_CAFs = expression[CAF_ids,] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}) %>% .[. >= 4] %>% names() 
expressed_genes_endothelial = expression[endothelial_ids,] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}) %>% .[. >= 4] %>% names() 
expressed_genes_malignant = expression[malignant_ids,] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}) %>% .[. >= 4] %>% names() 


 
### Load the ligand-target model we want to use 
 


In [ ]:
ligand_target_matrix = readRDS(url("https://zenodo.org/record/3260758/files/ligand_target_matrix.rds")) 
ligand_target_matrix[1:5,1:5] # target genes in rows, ligands in columns 
##                 CXCL1        CXCL2        CXCL3        CXCL5         PPBP 
## A1BG     3.534343e-04 4.041324e-04 3.729920e-04 3.080640e-04 2.628388e-04 
## A1BG-AS1 1.650894e-04 1.509213e-04 1.583594e-04 1.317253e-04 1.231819e-04 
## A1CF     5.787175e-04 4.596295e-04 3.895907e-04 3.293275e-04 3.211944e-04 
## A2M      6.027058e-04 5.996617e-04 5.164365e-04 4.517236e-04 4.590521e-04 
## A2M-AS1  8.898724e-05 8.243341e-05 7.484018e-05 4.912514e-05 5.120439e-05 


 
### Load the gene set of interest and background of genes 
 
As gene set of interest, we consider the genes of which the expression 
is possibly affected due to communication with other cells. 
 
Because we here want to investigate how CAFs and endothelial cells 
regulate the expression of p-EMT genes in malignant cells, we will use 
the p-EMT gene set defined by Puram et al. as gene set of interset and 
use all genes expressed in malignant cells as background of 
genes. 
 


In [ ]:
pemt_geneset = readr::read_tsv(url("https://zenodo.org/record/3260758/files/pemt_signature.txt"), col_names = "gene") %>% pull(gene) %>% .[. %in% rownames(ligand_target_matrix)] # only consider genes also present in the NicheNet model - this excludes genes from the gene list for which the official HGNC symbol was not used by Puram et al. 
head(pemt_geneset) 
## [1] "SERPINE1" "TGFBI"    "MMP10"    "LAMC2"    "P4HA2"    "PDPN" 
 
background_expressed_genes = expressed_genes_malignant %>% .[. %in% rownames(ligand_target_matrix)] 
head(background_expressed_genes) 
## [1] "RPS11"   "ELMO2"   "PNMA1"   "MMP2"    "TMEM216" "ERCC5" 


 
### Perform NicheNet’s ligand activity analysis on the gene set of interest 
 
In a first step, we will define a set of potentially active ligands. As 
potentially active ligands, we will use ligands that are 1) expressed by 
CAFs and/or endothelial cells and 2) can bind a (putative) receptor 
expressed by malignant cells. Putative ligand-receptor links were 
gathered from NicheNet’s ligand-receptor data sources. 
 
Note that we combine the ligands from CAFs and endothelial cells in one 
ligand activity analysis now. Later on, we will look which of the 
top-ranked ligands is mainly expressed by which of both cell 
types. 
 


In [ ]:
lr_network = readRDS(url("https://zenodo.org/record/3260758/files/lr_network.rds")) 
 
ligands = lr_network %>% pull(from) %>% unique() 
expressed_ligands_CAFs = intersect(ligands,expressed_genes_CAFs) 
expressed_ligands_endothelial = intersect(ligands,expressed_genes_endothelial) 
expressed_ligands = union(expressed_ligands_CAFs, expressed_genes_endothelial) 
 
receptors = lr_network %>% pull(to) %>% unique() 
expressed_receptors = intersect(receptors,expressed_genes_malignant) 
 
potential_ligands = lr_network %>% filter(from %in% expressed_ligands & to %in% expressed_receptors) %>% pull(from) %>% unique() 
head(potential_ligands) 
## [1] "IL15"    "HGF"     "TNFSF10" "TGFB2"   "TGFB3"   "INHBA" 


 
Now perform the ligand activity analysis: infer how well NicheNet’s 
ligand-target potential scores can predict whether a gene belongs to the 
p-EMT program or 
not. 
 


In [ ]:
ligand_activities = predict_ligand_activities(geneset = pemt_geneset, background_expressed_genes = background_expressed_genes, ligand_target_matrix = ligand_target_matrix, potential_ligands = potential_ligands) 



 
Now, we want to rank the ligands based on their ligand activity. In our 
validation study, we showed that the pearson correlation between a 
ligand’s target predictions and the observed transcriptional response 
was the most informative measure to define ligand activity. Therefore, 
we will rank the ligands based on their pearson correlation coefficient. 
 


In [ ]:
ligand_activities %>% arrange(-pearson)  
## # A tibble: 154 x 4 
##    test_ligand auroc   aupr pearson 
##    <chr>       <dbl>  <dbl>   <dbl> 
##  1 PTHLH       0.667 0.0720   0.128 
##  2 EDN1        0.682 0.0586   0.126 
##  3 CXCL12      0.680 0.0507   0.123 
##  4 AGT         0.676 0.0581   0.120 
##  5 TGFB3       0.689 0.0454   0.117 
##  6 IL6         0.693 0.0510   0.115 
##  7 INHBA       0.695 0.0502   0.113 
##  8 ADAM17      0.672 0.0526   0.113 
##  9 TNC         0.700 0.0444   0.109 
## 10 VWF         0.685 0.0490   0.109 
## # … with 144 more rows 
best_upstream_ligands = ligand_activities %>% top_n(20, pearson) %>% arrange(-pearson) %>% pull(test_ligand) 
head(best_upstream_ligands) 
## [1] "PTHLH"  "EDN1"   "CXCL12" "AGT"    "TGFB3"  "IL6" 


 
We see here that the top-ranked ligands can predict the p-EMT genes 
reasonably, this implies that ranking of the ligands might be accurate 
as shown in our study. However, it is possible that for some gene sets, 
the target gene prediction performance of the top-ranked ligands would 
not be much better than random prediction. In that case, prioritization 
of ligands will be less trustworthy. 
 
Determine now which prioritized ligands are expressed by CAFs and or 
endothelial cells 
 


In [ ]:
best_upstream_ligands %>% intersect(expressed_ligands_CAFs)  
##  [1] "PTHLH"  "CXCL12" "AGT"    "TGFB3"  "IL6"    "INHBA"  "ADAM17" "TNC"    "CTGF"   "FN1"    "BMP5"   "IL24"   
## [13] "CXCL11" "MMP9"   "COL4A1" "PSEN1"  "CXCL9" 
best_upstream_ligands %>% intersect(expressed_ligands_endothelial) 
##  [1] "EDN1"   "CXCL12" "IL6"    "ADAM17" "VWF"    "CTGF"   "FN1"    "SPP1"   "CXCL11" "COL4A1" "PSEN1"  "CXCL9" 
 
# lot of overlap between both cell types in terms of expressed ligands 
# therefore, determine which ligands are more strongly expressed in which of the two 
ligand_expression_tbl = tibble( 
  ligand = best_upstream_ligands,  
  CAF = expression[CAF_ids,best_upstream_ligands] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}), 
  endothelial = expression[endothelial_ids,best_upstream_ligands] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)})) 
 
CAF_specific_ligands = ligand_expression_tbl %>% filter(CAF > endothelial + 2) %>% pull(ligand) 
endothelial_specific_ligands = ligand_expression_tbl %>% filter(endothelial > CAF + 2) %>% pull(ligand) 
general_ligands = setdiff(best_upstream_ligands,c(CAF_specific_ligands,endothelial_specific_ligands)) 
 
ligand_type_indication_df = tibble( 
  ligand_type = c(rep("CAF-specific", times = CAF_specific_ligands %>% length()), 
                  rep("General", times = general_ligands %>% length()), 
                  rep("Endothelial-specific", times = endothelial_specific_ligands %>% length())), 
  ligand = c(CAF_specific_ligands, general_ligands, endothelial_specific_ligands)) 


 
### Infer target genes of top-ranked ligands and visualize in a circos plot 
 
Now we will show how you can look at the regulatory potential scores 
between ligands and target genes of interest. In this case, we will look 
at links between top-ranked p-EMT-regulating ligands and p-EMT genes. In 
this example, inferred target genes should belong to the p-EMT gene set 
and to the 250 most strongly predicted targets of at least one of the 
selected top-ranked ligands (the top 250 targets according to the 
general prior model, so not the top 250 targets for this dataset). 
 
Get first the active ligand-target links by looking which of the p-EMT 
genes are among the top-predicted target genes for the prioritized 
ligands: 
 


In [ ]:
active_ligand_target_links_df = best_upstream_ligands %>% lapply(get_weighted_ligand_target_links,geneset = pemt_geneset, ligand_target_matrix = ligand_target_matrix, n = 250) %>% bind_rows() 
 
active_ligand_target_links_df = active_ligand_target_links_df %>% mutate(target_type = "p_emt") %>% inner_join(ligand_type_indication_df) # if you want ot make circos plots for multiple gene sets, combine the different data frames and differentiate which target belongs to which gene set via the target type 


 
To avoid making a circos plots with too many ligand-target links, we 
will show only links with a weight higher than a predefined cutoff: 
links belonging to the 66% of lowest scores were removed. Not that this 
cutoffs and other cutoffs used for this visualization can be changed 
according to the user’s 
needs. 
 


In [ ]:
cutoff_include_all_ligands = active_ligand_target_links_df$weight %>% quantile(0.66) 
 
active_ligand_target_links_df_circos = active_ligand_target_links_df %>% filter(weight > cutoff_include_all_ligands) 
 
ligands_to_remove = setdiff(active_ligand_target_links_df$ligand %>% unique(), active_ligand_target_links_df_circos$ligand %>% unique()) 
targets_to_remove = setdiff(active_ligand_target_links_df$target %>% unique(), active_ligand_target_links_df_circos$target %>% unique()) 
   
circos_links = active_ligand_target_links_df %>% filter(!target %in% targets_to_remove &!ligand %in% ligands_to_remove) 


 
Prepare the circos visualization: give each segment of ligands and 
targets a specific color and order 
 


In [ ]:
grid_col_ligand =c("General" = "lawngreen", 
            "CAF-specific" = "royalblue", 
            "Endothelial-specific" = "gold") 
grid_col_target =c( 
            "p_emt" = "tomato") 
 
grid_col_tbl_ligand = tibble(ligand_type = grid_col_ligand %>% names(), color_ligand_type = grid_col_ligand) 
grid_col_tbl_target = tibble(target_type = grid_col_target %>% names(), color_target_type = grid_col_target) 
 
circos_links = circos_links %>% mutate(ligand = paste(ligand," ")) # extra space: make a difference between a gene as ligand and a gene as target! 
circos_links = circos_links %>% inner_join(grid_col_tbl_ligand) %>% inner_join(grid_col_tbl_target) 
links_circle = circos_links %>% select(ligand,target, weight) 
 
ligand_color = circos_links %>% distinct(ligand,color_ligand_type) 
grid_ligand_color = ligand_color$color_ligand_type %>% set_names(ligand_color$ligand) 
target_color = circos_links %>% distinct(target,color_target_type) 
grid_target_color = target_color$color_target_type %>% set_names(target_color$target) 
 
grid_col =c(grid_ligand_color,grid_target_color) 
 
# give the option that links in the circos plot will be transparant ~ ligand-target potential score 
transparency = circos_links %>% mutate(weight =(weight-min(weight))/(max(weight)-min(weight))) %>% mutate(transparency = 1-weight) %>% .$transparency  


 
Prepare the circos visualization: order ligands and targets 
 


In [ ]:
target_order = circos_links$target %>% unique() 
ligand_order = c(CAF_specific_ligands,general_ligands,endothelial_specific_ligands) %>% c(paste(.," ")) %>% intersect(circos_links$ligand) 
order = c(ligand_order,target_order) 


 
Prepare the circos visualization: define the gaps between the different 
segments 
 


In [ ]:
width_same_cell_same_ligand_type = 0.5 
width_different_cell = 6 
width_ligand_target = 15 
width_same_cell_same_target_type = 0.5 
 
gaps = c( 
  # width_ligand_target, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "CAF-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "General") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "Endothelial-specific") %>% distinct(ligand) %>% nrow() -1)),  
  width_ligand_target, 
  rep(width_same_cell_same_target_type, times = (circos_links %>% filter(target_type == "p_emt") %>% distinct(target) %>% nrow() -1)), 
  width_ligand_target 
  ) 


 
Render the circos plot (all links same transparancy). Only the widths of 
the blocks that indicate each target gene is proportional the 
ligand-target regulatory potential (\~prior knowledge supporting the 
regulatory interaction). 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = 0, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 1) 
}, bg.border = NA) # 


 
![](circos_files/figure-gfm/unnamed-chunk-15-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Render the circos plot (degree of transparancy determined by the 
regulatory potential value of a ligand-target interaction) 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 1) 
}, bg.border = NA) # 


 
![](circos_files/figure-gfm/unnamed-chunk-16-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Save circos plot to an svg file 
 


In [ ]:
svg("ligand_target_circos.svg", width = 10, height = 10) 
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid", 
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 1) 
}, bg.border = NA) # 
circos.clear() 
dev.off() 
## png  
##   2 


 
### Visualize ligand-receptor interactions of the prioritized ligands in a circos plot 
 


In [ ]:
# get the ligand-receptor network of the top-ranked ligands 
lr_network_top = lr_network %>% filter(from %in% best_upstream_ligands & to %in% expressed_receptors) %>% distinct(from,to) 
best_upstream_receptors = lr_network_top %>% pull(to) %>% unique() 
 
# get the weights of the ligand-receptor interactions as used in the NicheNet model 
weighted_networks = readRDS(url("https://zenodo.org/record/3260758/files/weighted_networks.rds")) 
lr_network_top_df = weighted_networks$lr_sig %>% filter(from %in% best_upstream_ligands & to %in% best_upstream_receptors) %>% rename(ligand = from, receptor = to) 
 
lr_network_top_df = lr_network_top_df %>% mutate(receptor_type = "p_emt_receptor") %>% inner_join(ligand_type_indication_df) 


In [ ]:
grid_col_ligand =c("General" = "lawngreen", 
            "CAF-specific" = "royalblue", 
            "Endothelial-specific" = "gold") 
grid_col_receptor =c( 
            "p_emt_receptor" = "darkred") 
 
grid_col_tbl_ligand = tibble(ligand_type = grid_col_ligand %>% names(), color_ligand_type = grid_col_ligand) 
grid_col_tbl_receptor = tibble(receptor_type = grid_col_receptor %>% names(), color_receptor_type = grid_col_receptor) 
 
circos_links = lr_network_top_df %>% mutate(ligand = paste(ligand," ")) # extra space: make a difference between a gene as ligand and a gene as receptor! 
circos_links = circos_links %>% inner_join(grid_col_tbl_ligand) %>% inner_join(grid_col_tbl_receptor) 
links_circle = circos_links %>% select(ligand,receptor, weight) 
 
ligand_color = circos_links %>% distinct(ligand,color_ligand_type) 
grid_ligand_color = ligand_color$color_ligand_type %>% set_names(ligand_color$ligand) 
receptor_color = circos_links %>% distinct(receptor,color_receptor_type) 
grid_receptor_color = receptor_color$color_receptor_type %>% set_names(receptor_color$receptor) 
 
grid_col =c(grid_ligand_color,grid_receptor_color) 
 
# give the option that links in the circos plot will be transparant ~ ligand-receptor potential score 
transparency = circos_links %>% mutate(weight =(weight-min(weight))/(max(weight)-min(weight))) %>% mutate(transparency = 1-weight) %>% .$transparency  


 
Prepare the circos visualization: order ligands and receptors 
 


In [ ]:
receptor_order = circos_links$receptor %>% unique() 
ligand_order = c(CAF_specific_ligands,general_ligands,endothelial_specific_ligands) %>% c(paste(.," ")) %>% intersect(circos_links$ligand) 
order = c(ligand_order,receptor_order) 


 
Prepare the circos visualization: define the gaps between the different 
segments 
 


In [ ]:
width_same_cell_same_ligand_type = 0.5 
width_different_cell = 6 
width_ligand_receptor = 15 
width_same_cell_same_receptor_type = 0.5 
 
gaps = c( 
  # width_ligand_receptor, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "CAF-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "General") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "Endothelial-specific") %>% distinct(ligand) %>% nrow() -1)),  
  width_ligand_receptor, 
  rep(width_same_cell_same_receptor_type, times = (circos_links %>% filter(receptor_type == "p_emt_receptor") %>% distinct(receptor) %>% nrow() -1)), 
  width_ligand_receptor 
  ) 


 
Render the circos plot (all links same transparancy). Only the widths of 
the blocks that indicate each receptor is proportional the 
ligand-receptor interaction weight (\~prior knowledge supporting the 
interaction). 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = 0, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 0.8) 
}, bg.border = NA) # 


 
![](circos_files/figure-gfm/unnamed-chunk-22-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Render the circos plot (degree of transparancy determined by the prior 
interaction weight of the ligand-receptor interaction - just as the 
widths of the blocks indicating each receptor) 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 0.8) 
}, bg.border = NA) # 


 
![](circos_files/figure-gfm/unnamed-chunk-23-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Save circos plot to an svg file 
 


In [ ]:
svg("ligand_receptor_circos.svg", width = 15, height = 15) 
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid", 
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 0.8) 
}, bg.border = NA) # 
circos.clear() 
dev.off() 
## png  
##   2 


 
### Remark on making a ligand-receptor-target circos plot 
 
In the paper of Bonnardel, T’Jonck et al. [Stellate Cells, Hepatocytes, 
and Endothelial Cells Imprint the Kupffer Cell Identity on Monocytes 
Colonizing the Liver Macrophage 
Niche](https://www.cell.com/immunity/fulltext/S1074-7613\(19\)30368-1), 
we showed in Fig. 6B a ligand-receptor-target circos plot to visualize 
the main NicheNet predictions. This “ligand-receptor-target” circos plot 
was made by making first two separate circos plots: the ligand-target 
and ligand-receptor circos plot. Then these circos plots were overlayed 
in Inkscape (with the center of the two circles at the same location and 
the ligand-receptor circos plot bigger than the ligand-target one). To 
generate the combined circos plot as shown ni Fig. 6B, we then manually 
removed all elements of the ligand-receptor circos plot except the outer 
receptor layer. In the near future, we will be working on a solution to 
generate this ligand-receptor-target circos plot in a fully automated 
manner. 
 
If you would want to split up target genes and receptors in different 
groups according to signaling pathway (as done in mentioned paper), then 
you first need to define these groups in a specific data frame in 
advance (cf what is shown for ligands in the 
`ligand_type_indication_df`in the vignette). When you then want to 
overlay receptors in this case, you need to make sure that the 
ligand-receptor weights of receptors in one group are proportional to 
the ligand-target weights of the targets in that group (to generate the 
nice overlay effect). So in that case, the ligand-receptor weights are 
proportional to the ‘underlying’ ligand-target regulatory potential 
scores and not reflective of prior information supporting the specific 
ligand-receptor interaction (as shown in this vignette for 
ligand-receptor circos plots). 
 
### References 
 
Bonnardel et al., 2019, Immunity 51, 1–17, [Stellate Cells, Hepatocytes, 
and Endothelial Cells Imprint the Kupffer Cell Identity on Monocytes 
Colonizing the Liver Macrophage 
Niche](https://doi.org/10.1016/j.immuni.2019.08.017) 
 
<div id="refs" class="references"> 
 
<div id="ref-puram_single-cell_2017"> 
 
Puram, Sidharth V., Itay Tirosh, Anuraag S. Parikh, Anoop P. Patel, 
Keren Yizhak, Shawn Gillespie, Christopher Rodman, et al. 2017. 
“Single-Cell Transcriptomic Analysis of Primary and Metastatic Tumor 
Ecosystems in Head and Neck Cancer.” *Cell* 171 (7): 1611–1624.e24. 
<https://doi.org/10.1016/j.cell.2017.10.044>. 
 
</div> 
 
</div> 
